# Model Tests

Repo: https://github.com/qubvel/segmentation_models.pytorch

Models: https://smp.readthedocs.io/en/latest/models.html#unet

# Imports

In [1]:
from segmentation.helper_functions.data_functions import prep_dataloader

C:\Users\s2137314\Anaconda3\envs\gel_env\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


# Create Segmentation Model

In [3]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
    activation='softmax'
)

preprocessing_fn = smp.encoders.get_preprocessing_fn('resnet18', pretrained='imagenet')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to C:\Users\s2137314/.cache\torch\hub\checkpoints\resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

# Loading Data

In [4]:
dir_img = 'C:/2022_Summer_Intern/Gel_Images_UNet_Test/Images_all_nathan+selected'
dir_mask = 'C:/2022_Summer_Intern/Gel_Images_UNet_Test/Masks_all_nathan+selected'
n_channels = 1
img_scale = 0.5
val_percent = 0.1
batch_size = 1
num_workers = 1

In [5]:
train_loader, val_loader, n_train, n_val = prep_dataloader(dir_img, 
                                                           dir_mask, 
                                                           n_channels, 
                                                           img_scale, 
                                                           val_percent, 
                                                           batch_size, 
                                                           num_workers)

# Do we need augmentation?

# Training

In [6]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

AttributeError: module 'segmentation_models_pytorch' has no attribute 'utils'

In [ ]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')